# Text Normalization (Chinese)

- `text_normalizer_zh.py`
- Including functions for:
    - word-seg chinese texts
    - clean up texts by removing duplicate spaces and line breaks
    - remove incompatible weird characters

In [1]:
import unicodedata
import re
#from nltk.corpus import wordnet
#import collections
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
import requests
import pandas as pd
import text_normalizer_zh as tnz

## Normalization Functions

In [2]:
# %load text_normalizer_zh.py
"""

Notes
-----

    These functions are based on the text normalization functions 
    provided in Text Analytics with Python 2ed.

"""

import unicodedata
import re
# from nltk.tokenize.toktok import ToktokTokenizer
import pandas as pd
import jieba

## Initialize Trad Chinese dictionary
jieba.set_dictionary('../../../RepositoryData/data/jiaba/dict.txt.jiebatw.txt')


## Normalize unicode characters
def remove_weird_chars(text):
    #     ```
    #     (NFKD) will apply the compatibility decomposition, i.e.
    #     replace all compatibility characters with their equivalents.
    #     ```
    text = unicodedata.normalize('NFKD', text).encode('utf-8',
                                                      'ignore').decode(
                                                          'utf-8', 'ignore')
    return text


## Remove extra linebreaks
def remove_extra_linebreaks(text):
    lines = text.split(r'\n+')
    return '\n'.join(
        [re.sub(r'[\s]+', ' ', l).strip() for l in lines if len(l) != 0])


## Remove extra medial/trailing/leading spaces
def remove_extra_spaces(text):
    return re.sub("\\s+", " ", text).strip()


## Seg the text into words
def seg(text):
    text_seg = jieba.cut(text)
    out = ' '.join(text_seg)
    return out


## Remove punctuation/symbols
def remove_symbols(text):
    """
    
    Unicode 6.0 has 7 character categories, and each category has subcategories:

    Letter (L): lowercase (Ll), modifier (Lm), titlecase (Lt), uppercase (Lu), other (Lo)
    Mark (M): spacing combining (Mc), enclosing (Me), non-spacing (Mn)
    Number (N): decimal digit (Nd), letter (Nl), other (No)
    Punctuation (P): connector (Pc), dash (Pd), initial quote (Pi), final quote (Pf), open (Ps), close (Pe), other (Po)
    Symbol (S): currency (Sc), modifier (Sk), math (Sm), other (So)
    Separator (Z): line (Zl), paragraph (Zp), space (Zs)
    Other (C): control (Cc), format (Cf), not assigned (Cn), private use (Co), surrogate (Cs)
    
    
    There are 3 ranges reserved for private use (Co subcategory): 
    U+E000—U+F8FF (6,400 code points), U+F0000—U+FFFFD (65,534) and U+100000—U+10FFFD (65,534). 
    Surrogates (Cs subcategory) use the range U+D800—U+DFFF (2,048 code points).
    
    
    """

    ## Brute-force version: list all possible unicode ranges, but this list is not complete.
    #   text = re.sub('[\u0021-\u002f\u003a-\u0040\u005b-\u0060\u007b-\u007e\u00a1-\u00bf\u2000-\u206f\u2013-\u204a\u20a0-\u20bf\u2100-\u214f\u2150-\u218b\u2190-\u21ff\u2200-\u22ff\u2300-\u23ff\u2460-\u24ff\u2500-\u257f\u2580-\u259f\u25a0-\u25ff\u2600-\u26ff\u2e00-\u2e7f\u3000-\u303f\ufe50-\ufe6f\ufe30-\ufe4f\ufe10-\ufe1f\uff00-\uffef─◆╱]+','',text)

    text = ''.join(ch for ch in text
                   if unicodedata.category(ch)[0] not in ['P', 'S'])
    return text


## Remove numbers
def remove_numbers(text):
    return re.sub('\\d+', "", text)


## Remove alphabets
def remove_alphabets(text):
    return re.sub('[a-zA-Z]+', '', text)


## Combine every step
def normalize_corpus(corpus,
                     is_remove_extra_linebreaks=True,
                     is_remove_weird_chars=True,
                     is_seg=True,
                     is_remove_symbols=True,
                     is_remove_numbers=True,
                     is_remove_alphabets=True):

    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:

        if is_remove_extra_linebreaks:
            doc = remove_extra_linebreaks(doc)

        if is_remove_weird_chars:
            doc = remove_weird_chars(doc)

        if is_seg:
            doc = seg(doc)

        if is_remove_symbols:
            doc = remove_symbols(doc)

        if is_remove_alphabets:
            doc = remove_alphabets(doc)

        if is_remove_numbers:
            doc = remove_numbers(doc)

        normalized_corpus.append(remove_extra_spaces(doc))

    return normalized_corpus

## Extract an Article

- Grab the first article from Google news for demonstration

In [3]:
url = 'https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNRFptTXpJU0JYcG9MVlJYS0FBUAE?hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
r = requests.get(url)
web_content = r.text
soup = BeautifulSoup(web_content, 'lxml')
title = soup.find_all('a', class_='DY5T1d')
first_art_link = title[0]['href'].replace('.', 'https://news.google.com', 1)

#print(first_art_link)
art_request = requests.get(first_art_link)
art_request.encoding = 'utf8'
soup_art = BeautifulSoup(art_request.text, 'lxml')

art_content = soup_art.find_all('p')
art_texts = [p.text for p in art_content]
print(art_texts)

['', '\n', '\n', '\n', '\r\n國民黨台南市議員謝龍介和民進黨台中市議員謝志忠，今年年初因為立法委員選情互賭，謝龍介賭輸，今天到台中市豐原葫蘆墩公園準備跳河履行承諾。', '\n', '\r\n他今天在跳水前接受媒體採訪，針對NCC昨天公聽會要審查中天換照公聽會，謝龍介，不要有用民粹殺死媒體 ，他這等於是公審方式，對任何媒體都不可以這樣做，他說每個媒體可能都不盡大家滿意，也不可能和所有人民立場一樣。', '\n', '\r\n謝龍介說， 這種公聽會方式為了服務執政黨，不符合言論自由的真義，他說昨天的公聽會是錯誤的示範， 否則將來每個媒體要換照是否都要直播， 讓立場不同的人來撻伐。', '\n', '\r\n謝龍介認為媒體應該回歸市場機制 ，讓市場決定媒體是否有能力存在經營， 他說這樣民主發展才可長可久， 這是我們奮鬥幾十年來得來不易的言論自由，他要NCC緊急煞車。台南市議員謝龍介今天下午，要在台中市豐原葫蘆墩公園跳水還賭約。記者游振昇/攝影', '\n', '\n', '\n                    國民黨台南市議員謝龍介和民進黨台中市議員謝志忠，今年年初因為立法委員選情互賭，謝龍介賭輸，今天到台中市豐原葫蘆墩公園準備...                  ', '\n                    針對10月26日中天新聞台換照聽證會，前立委孫大千在臉書表示，以聽證內容來看，顯然鑑定人已經有了預設立場，這根本就是一場...                  ', '\n                    國家通訊傳播委員會昨天舉辦中天新聞台換照聽證會，7位受邀鑑定人一面倒認為中天有缺失。國民黨立委賴士葆在臉書批評，他同意台...                  ', '\n                    中天電視台換照議題，是否衝擊蔡政府？財團法人台灣民意基金會上午公布最新民調顯示，有三成三民眾樂見中天電視台被撤照，五成三...                  ', '\n                    民進黨台北市議員王世堅自振興三倍券後，屢屢槓上行政院長蘇貞昌，日前指蘇「不只想當行政院長」，這次中天換照案，王世堅也暗指...                  ', '\n         

- Normalized results:

In [4]:
tnz.normalize_corpus([' '.join(art_texts)])

Building prefix dict from /Users/Alvin/GoogleDrive/_MySyncDrive/RepositoryData/data/jiaba/dict.txt.jiebatw.txt ...


Loading model from cache /var/folders/n7/ltpzwx813c599nfxfb94s_640000gn/T/jieba.u1b52b47246a0f2e6497af6bbe107adac.cache


Loading model cost 0.572 seconds.


Prefix dict has been built successfully.


['國民黨 台南 市議員 謝龍介 和 民進黨 台中市 議員 謝志忠 今年 年初 因為 立法 委員 選情 互賭 謝龍介 賭輸 今天 到 台中市 豐原 葫蘆墩 公園 準備 跳河 履行 承諾 他 今天 在 跳水 前 接受 媒體 採訪 針對 昨天 公聽會 要 審查 中天 換照 公聽會 謝龍介 不要 有用 民粹 殺死 媒體 他 這 等於 是 公審 方式 對 任何 媒體 都 不可以 這樣 做 他 說 每個 媒體 可能 都 不盡 大家 滿意 也 不可能 和 所有 人民 立場 一樣 謝龍介 說 這種 公聽會 方式 為了 服務 執政黨 不 符合 言論 自由 的 真義 他 說 昨天 的 公聽會 是 錯誤 的 示範 否則 將來 每個 媒體 要 換照 是否 都 要 直播 讓 立場 不同 的 人來 撻伐 謝龍介 認為 媒體 應該 回歸 市場 機制 讓 市場 決定 媒體 是否 有 能力 存在 經營 他 說 這樣 民主 發展 才 可長 可久 這是 我們 奮鬥 幾十年 來得 來 不易 的 言論 自由 他 要 緊急 煞車 台南 市議員 謝龍介 今天 下午 要 在 台中市 豐原 葫蘆墩 公園 跳水 還賭 約 記者 游振 昇 攝影 國民黨 台南 市議員 謝龍介 和 民進黨 台中市 議員 謝志忠 今年 年初 因為 立法 委員 選情 互賭 謝龍介 賭輸 今天 到 台中市 豐原 葫蘆墩 公園 準備 針對 月 日 中天 新聞台 換照 聽證會 前立委 孫大千 在 臉書 表示 以 聽證 內容 來 看 顯然 鑑定人 已經 有 了 預設 立場 這 根本 就是 一場 國家 通訊 傳播 委員會 昨天 舉辦 中天 新聞台 換照 聽證會 位 受邀 鑑定人 一面倒 認為 中天 有 缺失 國民黨 立委 賴士葆 在 臉書 批評 他 同意 台 中天 電視台 換照 議題 是否 衝擊 蔡 政府 財團 法人 台灣 民意 基金會 上午 公布 最 新 民調 顯示 有三成 三 民眾 樂見 中天 電視台 被 撤照 五成 三 民進黨 台北 市議員 王世堅 自 振興 三倍券 後 屢屢 槓上 行政院長 蘇貞昌 日前 指蘇 不只 想當 行政院長 這次 中天 換照 案 王世堅 也 暗指 中天 這次 能否 順利 換照 其實 民進黨 政府 承受 極大 的 壓力 因為 關或 不 關閉 這家 新聞台 都 會 得罪 另 一群 人 所以 大家 都 在 猜 蔡 政府 最後